<html>
<h1> Bergfex webscraping Part 3: 
</h1> finding nearest known snow level + creating a column with snow alert
<body>
<br />
</body>
</html>

still to do:
- check the creation of snow_coordinates in notebook 2
- check if better data resolution for swiss_cities, if not add city_coordinates to data: makes the "check for yourself" useful
- add testing notebook for NB III to github? or add section to 'proof' it's working in final NB III
- the trail index has been changed to ID

In [49]:
import pandas as pd
import numpy as np

import shapely
from shapely.geometry import Point, MultiPoint
from shapely.ops import nearest_points
import geopy.distance

import matplotlib
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.pyplot as plt

In [50]:
# importing trail GPS data
trail= pd.read_csv('C:/Users/Malwina/Desktop/bergfex_snow_alert/data/trail_coords_df.csv')

# importing trail altitude data
trail_info = pd.read_csv('C:/Users/Malwina/Desktop/bergfex_snow_alert/data/activities.csv')

# joining tables trail and trail_info[max]: 
trail['max_alt'] = trail_info['max']

# importing snow level data merged with cities coordinates
snow= pd.read_csv('C:/Users/Malwina/Desktop/bergfex_snow_alert/data/snow_coordinates.csv', sep=';', decimal=',')
snow = snow.dropna()
# EPSG:4326/WGS84
# renaming the snow columns to later be merged with the coordinates of the gps df
snow = snow.rename(columns = {'lat':'snow_obs_lat', 'lng':'snow_obs_lng'})
# renaming columns for clarity when merging
snow = snow.rename(columns = {'location':'snow_obs_location', 'height_in_m':'snow_obs_alt'})
snow

,snow_obs_location,snow_obs_alt,snowlevel_in_cm,snow_obs_lat,snow_obs_lng
2,Aigle,381,0,46.319025,6.970566
3,Altdorf,449,0,46.880600,8.639400
4,Andermatt,2282,0,46.602389,8.611800
5,Appenzell,780,0,47.333300,9.416700
6,Beatenberg,1182,0,46.703541,7.809551
7,Binningen,316,0,47.537870,7.570988
8,Bivio,1856,0,46.467767,9.650451
9,Blatten,1535,1,46.420105,7.820000
11,Bosco Gurin,1480,0,46.316402,8.492653
12,Buchs-Suhr,387,0,47.165540,9.470859


In [20]:
snow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48 entries, 2 to 76
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   snow_obs_location  48 non-null     object 
 1   snow_obs_alt       48 non-null     int64  
 2   snowlevel_in_cm    48 non-null     int64  
 3   snow_obs_lat       48 non-null     float64
 4   snow_obs_lng       48 non-null     float64
dtypes: float64(2), int64(2), object(1)
memory usage: 2.2+ KB


In [51]:
trail

,trail_id,name,lat,lon,max_alt
0,586723,Rundtour auf abenteuerlichen Wegen,46.704277,8.335683,1659
1,50425,An stolzen Bauernsitzen vorüber durch den südl...,47.091503,7.791530,1542
2,348123,Meienberg – Rundwanderung Seebergsee - Meienberg,46.582542,7.423618,1397
3,162740,Höhenweg Entlebuch Emmental - Etappe 3: Turner...,46.935223,7.914489,1115
4,49031,Zur Aussichtswarte Lueg (Heiligenlandhubel),47.064472,7.658991,1037
5,48808,Bergsee-Paradies am Stockhorn,46.683945,7.542221,1118
6,247242,Cable car to Wispile. Walk to Lauenensee,46.439976,7.292400,2079
7,131629,Via Alpina: Gstaad - L'Etivaz,46.475567,7.283884,1443


#### <html>
<h2> Compare hike location and snow level location
</h2>


</html>

we want to calculate the closest point in that df to another gps point


In [52]:
# making a single multipoint object out of all the snow obs points 
# making Points out of the snow points

# creating a grid out of all the snow points in the df snow
snow_point_list= []

for index, row in snow.iterrows():
    point = Point(row['snow_obs_lat'], row['snow_obs_lng'])
    snow_point_list.append(point)
    
len(snow_point_list) # 48 locations 

snow_grid = MultiPoint(snow_point_list)

In [53]:
# finding which snow point is nearest to the test point 
nearest_list = [] # list that contains tuples of nearest snow point in Point object format
near_lng = []  # contains the longitudes of the nearest snow point extracted from nearest shapely calculation
near_lat = []  # latitudes of nearest snow point

for index, row in trail.iterrows():
    tested_point = Point(row['lat'], row['lon'])
    nearest = nearest_points(tested_point, snow_grid)
    nearest_list.append(nearest[1]) # the second item in the tuple is the nearest point

for i in range(len(nearest_list)):
    near_lng.append(nearest_list[i].y) 
    near_lat.append(nearest_list[i].x)

In [54]:
# test: dataframe output works for each row of trails
# should not return an error
assert len(near_lng) == len(trail)

In [55]:
# adding the coordinates of the nearest snow point to each row of the trail df
trail['snow_obs_lat'] = near_lat
trail['snow_obs_lng'] = near_lng

# now we need to match those calculated nearest coordinates with the name of the snow_point with same coordinates
# we merge the two dataframes on coordinates, with replicating the rows of snow points if necessary

# merged should have length of gps dataframe rows
merged = pd.merge(trail, snow, on=['snow_obs_lat','snow_obs_lng'])
merged

,trail_id,name,lat,lon,max_alt,snow_obs_lat,snow_obs_lng,snow_obs_location,snow_obs_alt,snowlevel_in_cm
0,586723,Rundtour auf abenteuerlichen Wegen,46.704277,8.335683,1659,46.790917,8.387700,Engelberg,1035,0
1,50425,An stolzen Bauernsitzen vorüber durch den südl...,47.091503,7.791530,1542,46.890466,7.821042,Trubschachen-Blapbach,1091,0
2,162740,Höhenweg Entlebuch Emmental - Etappe 3: Turner...,46.935223,7.914489,1115,46.890466,7.821042,Trubschachen-Blapbach,1091,0
3,348123,Meienberg – Rundwanderung Seebergsee - Meienberg,46.582542,7.423618,1397,46.472472,7.546399,Chuenisbaergli-Schneehoehe,0,3
4,48808,Bergsee-Paradies am Stockhorn,46.683945,7.542221,1118,46.472472,7.546399,Chuenisbaergli-Schneehoehe,0,3
5,49031,Zur Aussichtswarte Lueg (Heiligenlandhubel),47.064472,7.658991,1037,46.998256,7.451340,Zollikofen,553,0
6,247242,Cable car to Wispile. Walk to Lauenensee,46.439976,7.292400,2079,46.311858,7.482353,Crans-Montana,1427,0
7,131629,Via Alpina: Gstaad - L'Etivaz,46.475567,7.283884,1443,46.311858,7.482353,Crans-Montana,1427,0


# Claculate the snow alert

In [56]:
# We want to make sure our snow level is actually relevant: 
# Condition 1 : the snow point has to be less than 30 km away from the hike (30 km is in between the average resolution of the
# snow thickness maps of SLF, the hi res being 2-5 km, the larger one around 50)

# Calculating the distance between gps point and nearest point

distance = []  # contains the distances in km from snow measurment to hike start
for index, row in merged.iterrows():
    gps_coords = (row['lat'], row['lon'])
    snow_coords = (row['snow_obs_lat'], row['snow_obs_lng'])
    d = round(geopy.distance.geodesic(gps_coords, snow_coords).km, 1)
    distance.append(d)
    
merged['distance'] = distance
merged['snow_status'] = 'check for yourself' # starting with a default status

merged.loc[merged['distance'] > 30, 'snow_status'] = 'No local data'
merged


,trail_id,name,lat,lon,max_alt,snow_obs_lat,snow_obs_lng,snow_obs_location,snow_obs_alt,snowlevel_in_cm,distance,snow_status
0,586723,Rundtour auf abenteuerlichen Wegen,46.704277,8.335683,1659,46.790917,8.387700,Engelberg,1035,0,10.4,check for yourself
1,50425,An stolzen Bauernsitzen vorüber durch den südl...,47.091503,7.791530,1542,46.890466,7.821042,Trubschachen-Blapbach,1091,0,22.5,check for yourself
2,162740,Höhenweg Entlebuch Emmental - Etappe 3: Turner...,46.935223,7.914489,1115,46.890466,7.821042,Trubschachen-Blapbach,1091,0,8.7,check for yourself
3,348123,Meienberg – Rundwanderung Seebergsee - Meienberg,46.582542,7.423618,1397,46.472472,7.546399,Chuenisbaergli-Schneehoehe,0,3,15.4,check for yourself
4,48808,Bergsee-Paradies am Stockhorn,46.683945,7.542221,1118,46.472472,7.546399,Chuenisbaergli-Schneehoehe,0,3,23.5,check for yourself
5,49031,Zur Aussichtswarte Lueg (Heiligenlandhubel),47.064472,7.658991,1037,46.998256,7.451340,Zollikofen,553,0,17.4,check for yourself
6,247242,Cable car to Wispile. Walk to Lauenensee,46.439976,7.292400,2079,46.311858,7.482353,Crans-Montana,1427,0,20.4,check for yourself
7,131629,Via Alpina: Gstaad - L'Etivaz,46.475567,7.283884,1443,46.311858,7.482353,Crans-Montana,1427,0,23.8,check for yourself


In [57]:
# Condition 2 : # filter on altitude max <1000 m difference 
# We chose 1000 m because it's the resolution of SLF data
# if thre is snow less than 1000 m higher than the max alt of the hike, we say there is snow
merged.loc[(merged['snowlevel_in_cm'] > 0) & (merged['max_alt'] + 1000 > merged['snow_obs_alt']), 'snow_status'] = 'Snow'
# if snow > 0 but higher than 1000 m over the max, leave "check for yourself status"
#
# if there is no snow and the max hike alt is less than 1000 m over the measurement point: no snow
merged.loc[(merged['snowlevel_in_cm'] == 0) & (merged['max_alt'] <= merged['snow_obs_alt'] + 1000), 'snow_status'] = 'No Snow'
# # if there is no snow and the max hike alt is more than 1000 m over the measurement point: no data, check for yourself
merged.loc[(merged['snowlevel_in_cm'] == 0) & (merged['max_alt'] > merged['snow_obs_alt'] + 1000), 'snow_status'] = 'No local data'
# if any snow point in a radius of 60 km of a lower altitude has snow, we want a snow warning

# if the measurement is within 1000 m up and down of the highest point, we consider it accurate
merged

,trail_id,name,lat,lon,max_alt,snow_obs_lat,snow_obs_lng,snow_obs_location,snow_obs_alt,snowlevel_in_cm,distance,snow_status
0,586723,Rundtour auf abenteuerlichen Wegen,46.704277,8.335683,1659,46.790917,8.387700,Engelberg,1035,0,10.4,No Snow
1,50425,An stolzen Bauernsitzen vorüber durch den südl...,47.091503,7.791530,1542,46.890466,7.821042,Trubschachen-Blapbach,1091,0,22.5,No Snow
2,162740,Höhenweg Entlebuch Emmental - Etappe 3: Turner...,46.935223,7.914489,1115,46.890466,7.821042,Trubschachen-Blapbach,1091,0,8.7,No Snow
3,348123,Meienberg – Rundwanderung Seebergsee - Meienberg,46.582542,7.423618,1397,46.472472,7.546399,Chuenisbaergli-Schneehoehe,0,3,15.4,Snow
4,48808,Bergsee-Paradies am Stockhorn,46.683945,7.542221,1118,46.472472,7.546399,Chuenisbaergli-Schneehoehe,0,3,23.5,Snow
5,49031,Zur Aussichtswarte Lueg (Heiligenlandhubel),47.064472,7.658991,1037,46.998256,7.451340,Zollikofen,553,0,17.4,No Snow
6,247242,Cable car to Wispile. Walk to Lauenensee,46.439976,7.292400,2079,46.311858,7.482353,Crans-Montana,1427,0,20.4,No Snow
7,131629,Via Alpina: Gstaad - L'Etivaz,46.475567,7.283884,1443,46.311858,7.482353,Crans-Montana,1427,0,23.8,No Snow


In [ ]:
# Condition 2 : # filter on altitude max <1000 m difference 
# We chose 1000 m because it's the resolution of SLF data
# if the snow measurement was done more than 1000 m lower than the max alt of the hike, we want a warning saying potential snow
# if the measurement is more than 1000 m higher than the max alt, we want a message saying the hike is low altitude
# if any snow point in a radius of 80 km of a lower altitude has snow, we want a snow warning
# if the measurement is within 1000 m up and down of the highest point, we consider it accurate